In [1]:
import psycopg2 as pg
import pandas as pd
import numpy as np
import alpaca_trade_api as tradeapi
import psycopg2.extras
import json

In [2]:
cred_path = "credential/client_details.json"

In [3]:
credential = (json.load(open(cred_path)))

In [13]:
connection = pg.connect(
   database="postgres", user='postgres', password='admin', host='localhost', port= '5432'
)

In [14]:
cursor = connection.cursor()

In [6]:
api = tradeapi.REST(credential["client_id"], credential["client_secret"], base_url = "https://paper-api.alpaca.markets")

In [7]:
assets = api.list_assets()

### Dropping previous table if exist

In [15]:
cursor.execute("DROP TABLE IF EXISTS tickers")

### Create a new table to contain all the asset data

In [16]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS  tickers (
    symbol VARCHAR(255) NOT NULL, 
    name VARCHAR(255) NOT NULL,
    asset_class VARCHAR(255) NOT NULL,
    exchange VARCHAR(255) NOT NULL,
    tradable BOOLEAN NOT NULL,
    shortable BOOLEAN NOT NULL,
    marginable BOOLEAN NOT NULL,
    internal_id VARCHAR(255) PRIMARY KEY
);
""")

In [17]:
query = "INSERT INTO tickers (symbol, name, asset_class, exchange, tradable, shortable, marginable, internal_id) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

In [18]:
### class and id cannot be called as an attribute, hence using the __getattr__ method to fetch the data 
for asset in assets:
    cursor.execute(query, (asset.symbol, asset.name, asset.__getattr__("class"), asset.exchange, asset.tradable, asset.shortable, asset.marginable, asset.__getattr__("id")))

### Commiting the data and closing the connection

In [19]:
cursor.close()

connection.commit()

connection.close()

### Check if the data have been inserted into the table

In [21]:
connection = pg.connect(
   database="postgres", user='postgres', password='admin', host='localhost', port= '5432'
)
cursor = connection.cursor()

In [22]:
query = """
SELECT * FROM tickers
    LIMIT 50;
"""


In [23]:
cursor.execute(query)

In [24]:
cursor.fetchall()

[('CREAY',
  'Creative Technology Ltd. Unsponsored ADR (Singapore)',
  'us_equity',
  'OTC',
  False,
  False,
  False,
  'feb49a65-2488-4865-95d1-d2887979feb4'),
 ('AACAF',
  'AAC Technologies Holdings Inc Ordinary Shares (Cayman Islands)',
  'us_equity',
  'OTC',
  False,
  True,
  False,
  '410b2e16-c94e-43f4-b541-09fd790bfd3c'),
 ('AAFRF',
  'Airtel Africa Plc Ordinary Shares (United Kingdom)',
  'us_equity',
  'OTC',
  False,
  True,
  False,
  '1d32a532-384f-461f-a3ad-2c5f34b54fd4'),
 ('AAGFF',
  'Aftermath Silver Ltd Ordinary Shares',
  'us_equity',
  'OTC',
  False,
  False,
  False,
  '785d3c7b-bacb-4e74-822e-955a29a5208d'),
 ('AAIC-B',
  'Arlington Asset Investment Corp. 7.00% Series B Cumulative Perpetual Redeemable Preferred Stock',
  'us_equity',
  'NYSE',
  False,
  False,
  False,
  '5b9c4c16-d6ae-495f-b056-5cc50a33af18'),
 ('AAMMF',
  'Almadex Minerals Ltd Ordinary Shares (Canada)',
  'us_equity',
  'OTC',
  False,
  False,
  False,
  'cc960718-ec39-4838-86e1-a8fc6f3aa3

In [25]:
cursor.close()

In [26]:
connection.close()